# 人生计算器

In [2]:
import numpy as np
import altair as alt
import pandas as pd
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

In [19]:
def life_calculator(years, initial_savings, salary_func, investment_func, spending_func):
    """
    计算并返回未来几年的每年净收入，考虑资金累积。
    - years: 预测年数
    - initial_savings: 初始存款金额
    - salary_func: 接收年份，返回每年的税后收入数组
    - investment_func: 接收年份和上年末余额，返回每年的投资收入数组
    - spending_func: 接收年份，返回每年的花销数组
    """
    years_array = np.arange(1, years + 1)
    savings = np.zeros(years)  # 初始化一个存储每年末余额的数组
    net_income = np.zeros(years)
    investment_income = np.zeros(years)
    savings[0] = initial_savings  # 设定初始存款
    salary_arr = salary_func(years)
    spending_arr = spending_func(years)
    
    for i in range(years):
        year = years_array[i]
        salary = salary_arr[i]
        spending = spending_arr[i]
        
        previous_savings = savings[i - 1] if i > 0 else initial_savings
        investment_income[i] = investment_func(year, previous_savings)
        
        net_income[i] = salary + investment_income[i] - spending
        savings[i] = previous_savings + net_income[i]
    
    return years_array, net_income, savings, investment_income, salary_arr, spending_arr

In [29]:
def create_charts(years, net_income, total_savings, investment_income, salary_income, income_target, savings_target, investment_percentage_target):
  data = pd.DataFrame({
    'Year': years,
    'Net Income': net_income,
    'Total Savings': total_savings,
    'Investment Income': investment_income,
    'Salary Income': salary_income,
    'Investment Percentage': investment_income / (investment_income + salary_income) * 100  # Calculate investment income percentage
  })
  def find_first_year(data, column, threshold):
      """找到数据首次达到某一阈值的年份"""
      years = data['Year'][data[column] >= threshold]
      if len(years) > 0:
          return years.iloc[0]
      else:
          return None  # 如果没有达到目标，则返回 None
  income_target_year = find_first_year(data, 'Net Income', income_target)
  savings_target_year = find_first_year(data, 'Total Savings', savings_target)
  investment_percentage_target_year = find_first_year(data, 'Investment Percentage', investment_percentage_target)

  base = alt.Chart(data).encode(x='Year:O')

  # 净收入图
  net_income_chart = base.mark_line(color='blue', point=True).encode(
      y='Net Income:Q',
      tooltip=['Year', 'Net Income']
  ).properties(
      title="Net Income Over Years"
  )


  salary_line = base.mark_line(color='blue', point=True).encode(
      y='Salary Income:Q',
      tooltip=['Year', 'Salary Income']
  ).properties(
      title="Income Over Years",
      width=600,
      height=300
  )
  
  investment_line = base.mark_line(color='orange', point=True).encode(
      y='Investment Income:Q',
      tooltip=['Year', 'Investment Income']
  )

  # 总储蓄图
  savings_chart = base.mark_line(color='green', point=True).encode(
      y='Total Savings:Q',
      tooltip=['Year', 'Total Savings']
  ).properties(
      title="Total Savings Over Years"
  )

  # 创建投资收入占比图
  investment_percentage_chart = alt.Chart(data).mark_line(point=True).encode(
      x='Year:O',
      y=alt.Y('Investment Percentage:Q', title='Investment Income as Percentage of Total Income'),
      tooltip=['Year', 'Investment Percentage']
  ).properties(
      title="Investment Income Percentage Over Years",
      width=600,
      height=300
  )

  # 如果有达到目标的年份，则添加标记线
  if income_target_year:
      income_target_line = alt.Chart(pd.DataFrame({'Year': [income_target_year]})).mark_rule(color='red', strokeDash=[6,2]).encode(
          x='Year:O'
      )
      net_income_line = alt.Chart(pd.DataFrame({'y': [income_target]})).mark_rule(color='red', strokeDash=[6,2]).encode(
          y='y:Q'
      )
      net_income_chart += income_target_line + net_income_line
  if savings_target_year:
      savings_target_line = alt.Chart(pd.DataFrame({'Year': [savings_target_year]})).mark_rule(color='red', strokeDash=[6,2]).encode(
          x='Year:O'
      )
      # 添加总储蓄达到 10,000,000 美元的水平线
      savings_line = alt.Chart(pd.DataFrame({'y': [savings_target]})).mark_rule(color='red', strokeDash=[6,2]).encode(
          y='y:Q'
      )
      savings_chart += savings_target_line + savings_line
  if investment_percentage_target_year:
     investment_percentage_target_line = alt.Chart(pd.DataFrame({'Year': [investment_percentage_target_year]})).mark_rule(color='red', strokeDash=[6,2]).encode(
         x='Year:O'
     )
     investment_percentage_line = alt.Chart(pd.DataFrame({'y': [investment_percentage_target]})).mark_rule(color='red', strokeDash=[6,2]).encode(
         y='y:Q'
     )
     investment_percentage_chart += investment_percentage_target_line + investment_percentage_line



  return (net_income_chart | savings_chart) & (investment_percentage_chart | investment_line + salary_line) # 并排显示两个图表


In [52]:
def salary_func_china(years):
    # 假设起始年薪为税后120,000美元，并每年增长5%
    salary_arr = np.zeros(years)
    for i in range(years):
      salary_arr[i] = 340000 * 0.14 * (1.05 ** (i - 1))
    return salary_arr

def spending_func_china(years):
    # 假设每年的花销为20,000美元，并每年通货膨胀3%
    spending_arr = np.zeros(years)
    for i in range(years):
      spending_arr[i] = 100000 * 0.14 * (1.03 ** (i - 1))
    return spending_arr

def salary_func_us(years):
    # 假设起始年薪为税后120,000美元，并每年增长5%
    salary_arr = np.zeros(years)
    for i in range(years):
      if i == 0:
        salary_arr[i] = 120000
      elif i == 1:
         salary_arr[i] = salary_arr[i - 1] * 1.081 
      elif i >= 2 and i <= 5:
         salary_arr[i] = salary_arr[i - 1] * 1.15
      elif i >= 8 and i <= 9:
         salary_arr[i] = salary_arr[i - 1] * 1.08
      elif i >= 15 and i <= 16:
         salary_arr[i] = salary_arr[i - 1] * 1.08
      else:
        salary_arr[i] = salary_arr[i-1] * 1.03
    return salary_arr
def spending_func_us(years):
    # 假设每年的花销为20,000美元，并每年通货膨胀3%
    spending_arr = np.zeros(years)
    for i in range(years):
      spending_arr[i] = 100000 * (1.03 ** (i - 1))
    return spending_arr

In [53]:
@interact(investment_return_per_year=(0.0,1.0,0.01), salary_func=[('us', salary_func_us), ('cn', salary_func_china)], spending_func=[('us', spending_func_us), ('cn', spending_func_china)], initial_savings = (0,100000,10000))
def interactive_plot(salary_func, spending_func, investment_return_per_year = 0.07, initial_savings = 80000):
    def investment_func(year, previous_savings):
      # 假设每年的投资回报率为7%
      return previous_savings * investment_return_per_year
    # 示例函数

    income_target = 365000 * 3
    savings_target = 50000000 * 0.14
    investment_percentage_target = 90
    num_of_years = 30

    years, net_income, total_savings, investment_income, salary_income, spendings = life_calculator(num_of_years, initial_savings, salary_func, investment_func, spending_func)

    # 创建总储蓄和净收入图表
    chart = create_charts(years, net_income, total_savings, investment_income, salary_income, income_target, savings_target, investment_percentage_target)

    return chart

interactive(children=(Dropdown(description='salary_func', options=(('us', <function salary_func_us at 0x120135…